Update num_mutations_averted in recombinants.csv

Need to fetch k4_num_mutations and k1000_num_mutations

In [1]:
import json
from pathlib import Path
import pandas as pd

In [2]:
data_dir = Path("../data")

In [3]:
# Old file
# recombinant, sample_id
recomb_file = data_dir / "recombinants.csv"
recomb_df = pd.read_csv(recomb_file).sort_values(by="sample_id")
recomb_df.head(1)

,recombinant,descendants,sample,sample_id,sample_pango,interval_left,interval_right,num_mutations,date_added,group_id,...,parent_right_pango,mrca,t_mrca,diffs,max_run_length,k2_num_mutations,k1000_num_mutations,num_mutations_averted,max_runlength_3sep,likely_artifactual
113,414488,5,414486,DRR321268,XC,26768,27390,1,2021-08-13,c179e2745fee9355a686734447789a6e,...,B.1.1.7,59,1114.000001,80,8,1,15,14,3,False


In [4]:
# k=4
# n_mutations
match_file = data_dir / "v1-beta1.matches.tsv"
match_df = pd.read_csv(match_file, sep="\t")
match_df.head(1)

,strain,n_parents,n_mutations,parents,mutations
0,SRR14631544,1,24,"(0:29904, 1)","[C241T, C3037T, C3267T, C4683T, C5986T, C6471T..."


In [5]:
# k=1000
# here, cost = num mutations
rerun_file = data_dir / "recombinant_reruns.json"
data = []
with open(rerun_file, 'r') as f:
    for line in f:
        d = json.loads(line.strip())
        strain = d["strain"]
        # Skip k=2
        if d["num_mismatches"] != 1000:
            continue
        num_mutations = d["match"]["cost"]
        d = {
            "strain": strain,
            "num_mutations": num_mutations,
        }
        data.append(d)
rerun_df = pd.DataFrame(data)
rerun_df.head(1)

,strain,num_mutations
0,SRR20259474,5


In [6]:
data = []
for i, row in rerun_df.iterrows():
    strain = row.loc["strain"]
    # k=4 result from matches file
    k4_num_mutations = match_df[
        match_df["strain"] == strain
    ]["n_mutations"].iloc[0]
    # k=1000 result from rerun file
    k1000_num_mutations = row.loc["num_mutations"]
    num_mutations_averted = k1000_num_mutations - k4_num_mutations
    data.append(
        {
            "strain": strain,
            "k4_num_mutations": k4_num_mutations,
            "k1000_num_mutations": k1000_num_mutations,
            "num_mutations_averted": num_mutations_averted,
        }
    )
res_df = pd.DataFrame(data).sort_values(by="strain")
res_df.head(1)

,strain,k4_num_mutations,k1000_num_mutations,num_mutations_averted
824,DRR321268,1,15,14


In [7]:
all([
    a == b for a, b in zip(
        recomb_df["sample_id"].to_list(),
        res_df["strain"].to_list(),
    )
])

True

In [8]:
all([
    a == b for a, b in zip(
        recomb_df["k1000_num_mutations"].to_list(),
        res_df["k1000_num_mutations"].to_list(),
    )
])

True

In [9]:
new_recomb_df = recomb_df\
    .drop("k2_num_mutations", axis=1)\
    .drop("k1000_num_mutations", axis=1)\
    .drop("num_mutations_averted", axis=1)\
    .assign(k4_num_mutations=res_df["k4_num_mutations"].to_list())\
    .assign(k1000_num_mutations=res_df["k1000_num_mutations"].to_list())\
    .assign(num_mutations_averted=res_df["num_mutations_averted"].to_list())
new_recomb_df.columns

Index(['recombinant', 'descendants', 'sample', 'sample_id', 'sample_pango',
       'interval_left', 'interval_right', 'num_mutations', 'date_added',
       'group_id', 'parent_left', 'parent_left_pango', 'parent_right',
       'parent_right_pango', 'mrca', 't_mrca', 'diffs', 'max_run_length',
       'max_runlength_3sep', 'likely_artifactual', 'k4_num_mutations',
       'k1000_num_mutations', 'num_mutations_averted'],
      dtype='object')

In [10]:
for c in recomb_df.columns:
    if c == "k2_num_mutations" or c == "num_mutations_averted":
        continue
    assert all(
        [
            a == b for a, b in
            zip(
                recomb_df[c].to_list(),
                new_recomb_df[c].to_list(),
            )
        ]), f"{c}"

In [11]:
new_recomb_file = data_dir / "recombinants.csv"
new_recomb_df\
    .sort_values(by="recombinant")\
    .to_csv(new_recomb_file, index=False)